In [5]:
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

In [12]:
# Imports and functions
import os
import google.generativeai as genai
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb
from langchain.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [19]:
# Chroma setup
persistent_client = chromadb.PersistentClient(path="lex2")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

db = Chroma(
    client=persistent_client,
    collection_name="lex_pod",
    embedding_function=embeddings,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
/tmp/ipykernel_61516/1583620799.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


# Question Prompt

In [18]:
system_prompt_default = (
    "You are a conversational expert with the goal of providing"
    "a thought provoking and open-ended discussion question."
)

system_prompt_style = (
    "You are a conversational expert with the goal of providing"
    "an open-ended discussion question. You should use the following"
    "conversation snippits to direct the style and tone with which"
    "you ask the question"
    "\n\n"
    "{style_context}"
)

input_prompt_default = (
    "Create a single discussion question that must be related to {topic}. "
    "The question must only contain text wiht no markup, emojis or additional formatting"
)

input_prompt_context = (
    "Create a single discussion question that must be related to {topic}."
    "Use the following snippits gain insight to context of the topic "
    "and assist in the generation of a relevant question:"
    "{document_context}"
    "The question must only contain text wiht no markup, emojis or additional formatting"

)

input_prompt_attributes = (
    "Create a single discussion question that must be related to {topic}."
    "The question should additionaly have the following attributes: {attributes}."
    "The question must only contain text wiht no markup, emojis or additional formatting"
)

input_prompt_attributes_context = (
    "Create a single discussion question that must be related to {topic}."
    "The question should additionaly have the following attributes: {attributes}"
    "Use the following snippits gain insight to context of the topic "
    "and assist in the generation of a relevant question:"
    "{document_context}"
    "The question must only contain text wiht no markup, emojis or additional formatting"
)

"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_default),
        ("human", input_prompt_default),
    ]
)
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_style),
        ("human", input_prompt_attributes_context),
    ]
)
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5}
)

topic = "Cowboys"
style_context = format_docs(retriever.invoke("Cowboys"))
document_context = "Cowboys are from the wild west"
attributes = "Dark sad odd"

rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)
response = rag_chain.invoke({"topic":topic, "attributes":attributes, 
"style_context": style_context,"document_context":document_context})
print(response)

NameError: name 'db' is not defined

# Topic Prompt

In [60]:
class TopicResponse(BaseModel):
    topic1: str
    topic2: str
    topic3: str

parser = PydanticOutputParser(pydantic_object=TopicResponse)

prompt_default = PromptTemplate(
    template = ("Generate 3 simple few word topics relating to {topic} that should lead to thought provoking and open-ended disucssion"
    "The topics generated should be short single statements. "
    "Under no circumstances should they be a question. \n{format_instructions}"),
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

prompt_attributes = PromptTemplate(
    template = ("Generate 3 simple few word topics relating to {topic} that should lead to thought provoking and open-ended disucssion"
    "The topics generated should be short single statements. "
    "The topics should have the following attributes:{attributes}"
    "Under no circumstances should they be a question. \n{format_instructions}"),
    input_variables=["topic","attributes"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


prompt_attributes_context = PromptTemplate(
    template = ("Generate 3 simple few word topics relating to {topic} that should lead to thought provoking and open-ended disucssion"
    "The topics generated should be short single statements. "
    "The topics generated should draw from the following context for direction: {context}. "
    "Under no circumstances should they be a question. \n{format_instructions}"),
    input_variables=["topic","context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


prompt_attributes_context = PromptTemplate(
    template = ("Generate 3 simple few word topics relating to {topic} that should lead to thought provoking and open-ended disucssion"
    "The topics generated should be short single statements. "
    "The topics should have the following attributes:{attributes}"
    "The topics generated should draw from the following context for direction: {context}. "
    "Under no circumstances should they be a question. \n{format_instructions}"),
    input_variables=["topic", "context", "attributes"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)



topic = "Cowboys"
context = "Cowboys are from the wild west"
attributes = "Happy fun silly"

rag_chain = (
    prompt_attributes_context
    | llm
    | parser
)
rag_chain.invoke({"topic":topic, "attributes":attributes, 
"context": context})

TopicResponse(topic1='Cowboys and Sea Shanty', topic2='Pirate Cowboys', topic3='Cowboy Treasure Hunts')